In [1]:
import os
import numpy as np
from skimage import io
from skimage import transform
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
dataset_path = r"C:\Users\SRINI\Downloads\archive (2)\Train"
def load_dataset(dataset_path):
    data = []
    labels = []
    for class_folder in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_folder)
        if os.path.isdir(class_path):
            for image_file in os.listdir(class_path):
                image_path = os.path.join(class_path, image_file)
                image = io.imread(image_path)
                image = transform.resize(image, (32, 32))  
                data.append(image)
                labels.append(int(class_folder))
    return np.array(data), np.array(labels)
data, labels = load_dataset(dataset_path)
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)
print(f"Size of the training dataset: {len(X_train)} samples")
print(f"Size of the testing dataset: {len(X_test)} samples")
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(43, activation='softmax'))  
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
datagen = ImageDataGenerator(rotation_range=10, zoom_range=0.1, width_shift_range=0.1, height_shift_range=0.1)
datagen.fit(X_train)

Size of the training dataset: 31367 samples
Size of the testing dataset: 7842 samples


In [2]:
epochs = 10
history = model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=epochs, validation_data=(X_test, y_test))
for epoch, acc, val_acc in zip(range(1, epochs+1), history.history['accuracy'], history.history['val_accuracy']):
    print(f"Epoch {epoch}/{epochs} - Training Accuracy: {acc:.4f} - Validation Accuracy: {val_acc:.4f}")

Epoch 1/10
981/981 [==============================] - 84s 75ms/step - loss: 2.4131 - accuracy: 0.3137 - val_loss: 1.0787 - val_accuracy: 0.6787
Epoch 2/10
981/981 [==============================] - 70s 72ms/step - loss: 1.3697 - accuracy: 0.5519 - val_loss: 0.6391 - val_accuracy: 0.8031
Epoch 3/10
981/981 [==============================] - 68s 69ms/step - loss: 1.0425 - accuracy: 0.6519 - val_loss: 0.3918 - val_accuracy: 0.8878
Epoch 4/10
981/981 [==============================] - 66s 67ms/step - loss: 0.8404 - accuracy: 0.7195 - val_loss: 0.2802 - val_accuracy: 0.9223
Epoch 5/10
981/981 [==============================] - 67s 68ms/step - loss: 0.7119 - accuracy: 0.7648 - val_loss: 0.1863 - val_accuracy: 0.9439
Epoch 6/10
981/981 [==============================] - 65s 66ms/step - loss: 0.6244 - accuracy: 0.7937 - val_loss: 0.1441 - val_accuracy: 0.9638
Epoch 7/10
981/981 [==============================] - 64s 65ms/step - loss: 0.5568 - accuracy: 0.8161 - val_loss: 0.1178 - val_accuracy:

In [3]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
loss, accuracy = model.evaluate(X_test, y_test)
print(f"\nAccuracy on the testing dataset: {accuracy*100:.2f}%")
predictions = model.predict(X_test)
y_test_argmax = y_test.argmax(axis=1)
predictions_argmax = predictions.argmax(axis=1)
class_names=[str(label) for label in lb.classes_]
classification_report_output = classification_report(y_test_argmax, predictions_argmax, target_names=class_names)
print(f"\nClassification Report:\n{classification_report_output}")
conf_mat = confusion_matrix(y_test_argmax, predictions_argmax)
print("\nConfusion Matrix:")
print(conf_mat)

246/246 [==============================] - 4s 17ms/step - loss: 0.0662 - accuracy: 0.9841

Accuracy on the testing dataset: 98.41%
246/246 [==============================] - 4s 15ms/step

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.79      0.88        38
           1       0.98      0.98      0.98       496
           2       0.94      0.98      0.96       420
           3       0.99      0.92      0.96       294
           4       0.99      0.99      0.99       400
           5       0.92      0.90      0.91       364
           6       0.99      1.00      0.99        69
           7       0.94      1.00      0.97       313
           8       1.00      0.97      0.99       274
           9       1.00      1.00      1.00       277
          10       1.00      1.00      1.00       398
          11       0.99      1.00      0.99       261
          12       1.00      1.00      1.00       443
          13       1.00      1.0

In [4]:
import numpy as np
from tensorflow.keras.preprocessing import image
def load_and_preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(32, 32))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img, img_array
image_path = r"C:\Users\SRINI\Downloads\archive (2)\Test\12614.png"
input_image, input_image_array = load_and_preprocess_image(image_path)
predictions = model.predict(input_image_array)
predicted_class = np.argmax(predictions)
predicted_label = lb.classes_[predicted_class]
print(f"Predicted Class: {predicted_label}")

1/1 [==============================] - 0s 81ms/step
Predicted Class: 2
